# Import & Load Emergency Care Data (2008 - 2019)

### Config

In [1]:
import sys
import configparser
config = configparser.ConfigParser()
config.read_file(open('../../settings.ini'))

sys.path.insert(0, config.get('PATHS','libs_path'))
engine_path = config.get('DATABASE','engine_path')

### Main params

In [2]:
remote_path = ''
local_path = './data_final/'

import postgres #from local file postgres.py
from commons import inline_table_xml, download_file, download_zip_file, extract_zip_file, inline_dimension_xml #from local file commons.py

import json
import pandas as pd
from sqlalchemy import create_engine

### Load file

In [3]:
#Open unzipped file
df = pd.read_csv("./data_final/fact_emergency.csv", delimiter = ",", dtype = "int")
df.head()

,date_id,geography_id,emergency_id,age_id,total
0,1205,3,1,5,41
1,1205,3,2,5,0
2,1205,3,3,5,10
3,1205,3,4,5,20
4,1205,3,5,5,18


### Ingest

In [4]:
engine = create_engine(engine_path)
db = postgres.PostgresDriver(engine)
db.to_sql(df, 'emergency_care', 'fact_emergency')

DROP TABLE IF EXISTS emergency_care.fact_emergency;
CREATE TABLE "emergency_care"."fact_emergency" (
"date_id" INTEGER,
  "geography_id" INTEGER,
  "emergency_id" INTEGER,
  "age_id" INTEGER,
  "total" INTEGER
)
COPY "emergency_care"."fact_emergency" ("date_id","geography_id","emergency_id","age_id","total") FROM STDIN WITH CSV HEADER DELIMITER ',';


### Add Indices to foreign keys

In [5]:
engine.execute("""
CREATE INDEX fact_emergency_date ON emergency_care.fact_emergency ("date_id"); """)

engine.execute("""
CREATE INDEX fact_emergency_geography ON emergency_care.fact_emergency ("geography_id"); """)

engine.execute("""
CREATE INDEX fact_emergency_emergency ON emergency_care.fact_emergency ("emergency_id"); """)

engine.execute("""
CREATE INDEX fact_emergency_age ON emergency_care.fact_emergency ("age_id"); """)

### Dimensions Table

In [6]:
import pandas as pd

# DATE
date_df = pd.read_csv("./data_final/date_table.csv")
engine = create_engine(engine_path)
db = postgres.PostgresDriver(engine)
db.to_sql(date_df, 'emergency_care', 'date_dimension')

DROP TABLE IF EXISTS emergency_care.date_dimension;
CREATE TABLE "emergency_care"."date_dimension" (
"date_id" INTEGER,
  "date_index" TEXT,
  "year_id" INTEGER,
  "quarter_id" INTEGER,
  "quarter_name" TEXT,
  "month_id" INTEGER,
  "month_name" TEXT,
  "week_id" INTEGER,
  "week_name" TEXT
)
COPY "emergency_care"."date_dimension" ("date_id","date_index","year_id","quarter_id","quarter_name","month_id","month_name","week_id","week_name") FROM STDIN WITH CSV HEADER DELIMITER ',';


In [7]:
# GEOGRAPHY
geography_df = pd.read_csv("./data_final/geography_table.csv")
engine = create_engine(engine_path)
db = postgres.PostgresDriver(engine)
db.to_sql(geography_df, 'emergency_care', 'geography_dimension')

DROP TABLE IF EXISTS emergency_care.geography_dimension;
CREATE TABLE "emergency_care"."geography_dimension" (
"geography_id" INTEGER,
  "geography_index" TEXT,
  "name" TEXT,
  "name_es" TEXT
)
COPY "emergency_care"."geography_dimension" ("geography_id","geography_index","name","name_es") FROM STDIN WITH CSV HEADER DELIMITER ',';


In [8]:
# EMERGENCY
emergency_df = pd.read_csv("./data_final/emergency_table.csv")
engine = create_engine(engine_path)
db = postgres.PostgresDriver(engine)
db.to_sql(emergency_df, 'emergency_care', 'emergency_dimension')

DROP TABLE IF EXISTS emergency_care.emergency_dimension;
CREATE TABLE "emergency_care"."emergency_dimension" (
"emergency_id" INTEGER,
  "emergency_index" TEXT,
  "name" TEXT,
  "name_es" TEXT,
  "cause_id" INTEGER,
  "cause" TEXT,
  "cause_es" TEXT,
  "action_id" INTEGER,
  "action" TEXT,
  "action_es" TEXT
)
COPY "emergency_care"."emergency_dimension" ("emergency_id","emergency_index","name","name_es","cause_id","cause","cause_es","action_id","action","action_es") FROM STDIN WITH CSV HEADER DELIMITER ',';


In [9]:
# AGE
age_df = pd.read_csv("./data_final/age_table.csv")
engine = create_engine(engine_path)
db = postgres.PostgresDriver(engine)
db.to_sql(age_df, 'emergency_care', 'age_dimension')

DROP TABLE IF EXISTS emergency_care.age_dimension;
CREATE TABLE "emergency_care"."age_dimension" (
"age_id" INTEGER,
  "age_index" TEXT,
  "name" TEXT,
  "name_es" TEXT
)
COPY "emergency_care"."age_dimension" ("age_id","age_index","name","name_es") FROM STDIN WITH CSV HEADER DELIMITER ',';


In [10]:
print("All done.")

All done.
